# Time series to supervised
Before machine learning can be used, time series forecasting problems must be re-framed as supervised learning problems. From a sequence to pairs of input and output sequences as described at https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

In [80]:
import boto3
import io
from sagemaker import get_execution_role

role = get_execution_role()
bucket ='sagemaker-data-energy-demand'

In [81]:
import numpy as np 
import pandas as pd 
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [82]:
def series_to_supervised(data,  col_names, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
     data: Sequence of observations as a list or NumPy array.
     n_in: Number of lag observations as input (X).
     n_out: Number of observations as output (y).
     dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
     Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (col_names[j], i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    if i == 0:
        names += [('%s(t)' % (col_names[j])) for j in range(n_vars)]
    else:
        names += [('%s(t+%d)' % (col_names[j], i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [83]:
CITY = 'LosAngeles'
df_file = 'dataframes/%s_selectedfeatures.csv' % CITY

df_location = 's3://{}/{}'.format(bucket, df_file)
df = pd.read_csv(df_location, index_col ='datetime')
len(list(df.columns))

25

In [86]:
cols = list(df.columns)
cols.remove('demand')
cols.insert(0,'demand')
df = df[cols]

values = df.values
# ensure all data is float
values = values.astype('float32')
# frame as supervised learning
reframed = series_to_supervised(values, df.columns, 1, 1)
reframed.drop(reframed.columns[[14,15,16,17,18,19,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35,36,37,38]], axis=1, inplace=True)

In [92]:
cols = list(reframed.columns)
cols.remove('demand(t)')
cols.insert(0,'demand(t)')
reframed = reframed[cols]
reframed.index = df.index[1:]

In [93]:
reframed.head()

,demand(t),demand(t-1),dailycoolingdegreedays(t-1),dailyheatingdegreedays(t-1),hourlydewpointtemperature(t-1),hourlydrybulbtemperature(t-1),hourlyprecipitation(t-1),hourlyrelativehumidity(t-1),hourlyvisibility(t-1),hourlywindspeed(t-1),...,dayofweek(t),quarter(t),month(t),year(t),dayofyear(t),dayofmonth(t),weekofyear(t),isbusinessday(t),isholiday(t),daylight(t)
datetime,,,,,,,,,,,,,,,,,,,,,
2015-07-01 09:00:00+00:00,3045.0,3298.0,11.0,0.0,63.0,74.0,0.0,69.0,10.0,0.0,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0
2015-07-01 10:00:00+00:00,2892.0,3045.0,11.0,0.0,64.0,77.0,0.0,64.0,10.0,0.0,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0
2015-07-01 11:00:00+00:00,2787.0,2892.0,11.0,0.0,63.0,82.0,0.0,53.0,10.0,0.0,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0
2015-07-01 12:00:00+00:00,2790.0,2787.0,11.0,0.0,61.0,84.0,0.0,46.0,10.0,3.0,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0
2015-07-01 13:00:00+00:00,2899.0,2790.0,11.0,0.0,61.0,81.0,0.0,51.0,10.0,3.0,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0


In [94]:
# save as csv file to continue in another notebook
s3_resource = boto3.resource('s3')

reframed_buffer = io.StringIO()
reframed_key = 'dataframes/%s_reframed.csv' % CITY
reframed.to_csv(reframed_buffer, compression=None)

s3_resource.Object(bucket, reframed_key).put(Body=reframed_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'C8CDDD9C36919D72',
  'HostId': '/f9XXe9ewF2VOUrFs8XJnsPJDQvOeK0Rdoll+W3rj/K9Uz7G2kHJ0yMgth8atSTuJdLJTrCYQxY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/f9XXe9ewF2VOUrFs8XJnsPJDQvOeK0Rdoll+W3rj/K9Uz7G2kHJ0yMgth8atSTuJdLJTrCYQxY=',
   'x-amz-request-id': 'C8CDDD9C36919D72',
   'date': 'Fri, 17 Jan 2020 03:48:53 GMT',
   'etag': '"4e64e21a6b342ffee5ee85e39efe5e95"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4e64e21a6b342ffee5ee85e39efe5e95"'}